In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import cx_Oracle
import configparser
import concurrent.futures
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display

```tnsnames.ora```

```ORCLCDB=(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=51521))
    (CONNECT_DATA=(SERVER=DEDICATED)(SERVICE_NAME=ORCLCDB.localdomain)))     
ORCLPDB1=(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=51521))
    (CONNECT_DATA=(SERVER=DEDICATED)(SERVICE_NAME=ORCLPDB1.localdomain)))```

In [2]:
host = 'localhost'
port = 51521
service_name = 'ORCLPDB1.localdomain'
user = r'test'
password = r'test'
dsn_tns = cx_Oracle.makedsn(host, port, service_name=service_name)

In [3]:
dsn = """
(DESCRIPTION =
    (ADDRESS = (PROTOCOL = TCP)(HOST = localhost)(PORT = 51521))
    (CONNECT_DATA =
      (SERVER = DEDICATED)
      (SERVICE_NAME = ORCLPDB1.localdomain)
    )
  )
"""

In [4]:
connection = None

try:
    connection = cx_Oracle.connect(user=user, password=password, dsn=dsn)
    print('Database version:', connection.version)
    print('Client version:', cx_Oracle.clientversion())
    print(connection.dsn)
except cx_Oracle.Error as error:
    print("Whoops!")
    print(err);
    sys.exit(1);

Database version: 12.2.0.1.0
Client version: (21, 6, 0, 0, 0)

(DESCRIPTION =
    (ADDRESS = (PROTOCOL = TCP)(HOST = localhost)(PORT = 51521))
    (CONNECT_DATA =
      (SERVER = DEDICATED)
      (SERVICE_NAME = ORCLPDB1.localdomain)
    )
  )



In [5]:
sql = """
SELECT
    id,
    age,
    sex,
    job,
    housing,
    saving_accounts,
    checking_account,
    credit_amount,
    duration,
    purpose
FROM
    german_credit_data
WHERE id BETWEEN :param1 AND :param2
ORDER BY
    id
"""

In [6]:
def Query(connection, sql, p1, p2):
    cursor = connection.cursor()
    cursor.execute(sql, param1 = p1, param2 = p2)
    
    column_list = []
    for column in cursor.description:
        column_list.append(column[0])
        
    data = []
    
    while True:
        row = cursor.fetchone()
        if row is None:
            break
        else:
            data.append(row)
            
    cursor.close()

    return pd.DataFrame(data, columns=column_list)

In [7]:
df = pd.DataFrame()

with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(Query, connection, sql, 0, 10)
    df = future.result()
    
display(df)

,ID,AGE,SEX,JOB,HOUSING,SAVING_ACCOUNTS,CHECKING_ACCOUNT,CREDIT_AMOUNT,DURATION,PURPOSE
0,0,67,male,2,own,NA,little,1169,6,radio/TV
1,1,22,female,2,own,little,moderate,5951,48,radio/TV
2,2,49,male,1,own,little,NA,2096,12,education
3,3,45,male,2,free,little,little,7882,42,furniture/equipment
4,4,53,male,2,free,little,little,4870,24,car
5,5,35,male,1,free,NA,NA,9055,36,education
6,6,53,male,2,own,quite rich,NA,2835,24,furniture/equipment
7,7,35,male,3,rent,little,moderate,6948,36,car
8,8,61,male,1,own,rich,NA,3059,12,radio/TV
9,9,28,male,3,own,little,moderate,5234,30,car


In [8]:
sql = """
SELECT
    id,
    age,
    sex,
    job,
    housing,
    saving_accounts,
    checking_account,
    credit_amount,
    duration,
    purpose
FROM
    german_credit_data
WHERE id BETWEEN 0 AND 10
ORDER BY
    id
"""

In [9]:
df = pd.read_sql(sql, con=connection)

In [10]:
display(df)

,ID,AGE,SEX,JOB,HOUSING,SAVING_ACCOUNTS,CHECKING_ACCOUNT,CREDIT_AMOUNT,DURATION,PURPOSE
0,0,67,male,2,own,NA,little,1169,6,radio/TV
1,1,22,female,2,own,little,moderate,5951,48,radio/TV
2,2,49,male,1,own,little,NA,2096,12,education
3,3,45,male,2,free,little,little,7882,42,furniture/equipment
4,4,53,male,2,free,little,little,4870,24,car
5,5,35,male,1,free,NA,NA,9055,36,education
6,6,53,male,2,own,quite rich,NA,2835,24,furniture/equipment
7,7,35,male,3,rent,little,moderate,6948,36,car
8,8,61,male,1,own,rich,NA,3059,12,radio/TV
9,9,28,male,3,own,little,moderate,5234,30,car


In [11]:
connection.close()